整体流程

1. 把你的内容拆成一块块的小文件块、对块进行了Embedding后放入向量库索引 （为后面提供语义搜索做准备）。

2. 搜索的时候把Query进行Embedding后通过语义检索找到最相似的K个Docs。

3. 把相关的Docs组装成Prompt的Context，基于相关内容进行QA，让GPT进行In Context Learning，用人话回答问题。

## 1. 环境准备

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "key" # 填入自己的 OpenAI Key
os.environ["HTTP_PROXY"] = "127.0.0.1:31426"
os.environ["HTTPS_PROXY"] = "127.0.0.1:31426"

## 2. LLM测试

创建 llm

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", temperature=0.9)

简单问答

In [3]:
query = "Please introduce yourself."
result = llm(query)

In [4]:
print(result)



I am an experienced software scientist and, currently, a full-time professor at Southern Illinois University Carbondale. I have been working on software development tools and technologies since the early 1990s, and have had a variety of experience with versions of C++ and Java. I am currently working on a tool to improve the quality of software by increasing the number ofuggets of code that it can run.


创建 PromptTemplate

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["chara"],
    template="Introduce yourself as {chara}",
)

query = prompt.format(chara="a musician")

In [6]:
query

'Introduce yourself as a musician'

In [7]:
result = llm(query)

In [8]:
result

'\n\nI am a musician and percussionist. I have been playing music for over 10 years.'

创建 chain

In [9]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run("the Dragonborn of The Elder Scrolls")

In [10]:
print(result)

 V: Dragonbulb

I am the Dragonborn of the Elder Scrolls V: Dragonbulb. I am a passenger in my tub, reading the instructions that my FryPy technician gave me.


## 3. ChatModel测试

创建 ChatModel

In [11]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

创建上下文信息

In [12]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [13]:
messages = [HumanMessage(content="请介绍一下你自己.")]
result = chat(messages)

In [14]:
result

AIMessage(content='我是一名AI语言模型，由OpenAI开发。我没有自己的个人信息和个性，但我可以根据用户输入的问题或指令进行回复和执行相应的任务。我可以帮助用户回答问题、提供信息、执行计算等各种任务。希望我能对您有所帮助。', additional_kwargs={}, example=False)

加入系统信息

In [15]:
messages = [
    SystemMessage(content="You are the Dragonborn in The Elder Scrolls."),
    HumanMessage(content="请介绍一下你自己.")
]
result = chat(messages)

In [16]:
result

AIMessage(content='我是一名虚构的角色，被称为“龙裔”，是《上古卷轴》系列游戏中的主角之一。我是一位拥有龙之血统的英雄，能够使用龙语，掌握龙吼和龙咆哮等强大的法术和技能。我的任务是拯救天际省（Skyrim）并击败龙神奥杜因（Alduin），同时探索广阔的大陆，完成各种任务和冒险。我可以选择不同的职业和技能，如战士、法师、盗贼，使用各种武器和魔法，与各种怪物和敌人战斗。我的旅程充满了危险和挑战，但我会努力成为天际省的英雄，赢得人们的尊敬和荣耀。', additional_kwargs={}, example=False)

## 4. 载入PDF

In [17]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("KOS：2023中国市场招聘趋势.pdf")
splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 400,
    chunk_overlap  = 100,
    length_function = len,
)
pages = loader.load_and_split(splitter)

In [18]:
pages[5]

Document(page_content='2022\nTechnology\nI. 出海科技\n2\n过往一年，国内经济市场处于疫情三年最低迷不振的时期，以互联网大厂\n为首的多梯度企业，纷纷做出业务收窄和人员优化的举措，进入保存核心\n业务实力和最优成本配置的企业防守阶段。纵然众多行业进入了防守阶\n段，出海科技行业却依旧在其中取得了较为亮眼的发展。\n改革开放伊始，从以“货”置“货”，到“制造代工”，到现在以“科\n技”、“设计”原创为主的科技输出，出海科技企业长足稳健的发展，得\n益于两个主要原因：\n1. 国内业务增量逐渐减缓，海外市场拓展或成新的机会“蛋糕”。\n纵观国内互联网发展，中国市场在过去十五年里，经历了高速蓬勃的发\n展。据公开数据可知，截止到2021年中国网民数达10亿左右，互联网渗透\n率虽高达7.0%，同比增长率却仅为4.34%。对比2020年，增速下滑5%。\n国内互联网产业趋于成熟饱和，红海市场无法逆转，因此如何在当下时代', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf', 'page': 4})

## 5. 文档存储

### OpenAI Embedding（二选一）

In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

指定 embedding 模型

In [20]:
embedding = OpenAIEmbeddings()
persist_directory = 'db_openai'

### 本地 Embedding（二选一）

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
embedding = HuggingFaceEmbeddings(model_name='text2vec-base-chinese')
persist_directory = 'db_huggingface'

如果先持久化 OpenAI 目录，再用 huggingface 模型写入相同目录，则会报错

In [ ]:
# persist_directory = 'db_openai'

### 文档持久化

从 Document 构建数据库，并持久化保存

In [23]:
vectordb = Chroma.from_documents(
    documents=pages, 
    embedding=embedding, 
    persist_directory=persist_directory)
vectordb.persist()

Using embedded DuckDB with persistence: data will be stored in: db_openai


## 6. 文档读取

清理之前的 vectordb，重新从本地读取

In [24]:
vectordb = None

retriever 用于根据索引查询向量数据库

搜索出 k 个最相关的 Document

In [25]:
vectordb = Chroma(persist_directory='db_openai', 
                  embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

Using embedded DuckDB with persistence: data will be stored in: db_openai


In [26]:
query = '2022年国内新能源车的渗透率是多少?'
docs = retriever.get_relevant_documents(query)

In [27]:
docs

[Document(page_content='水平的技术专家为主。\n国外市场在全球大力发展清洁能源的大环境下，全球各国均提出了大规模\n的新能源装机目标，储能行业同样受到了多国政府的激励。同时受俄乌冲\n突影响，欧洲户用储能和商用储能需求激增，不少企业储能出海业务需求\n和人才需求也维持在高位。 原来便携式储能出海产品在2021年火热，但\n在2022年则日趋红海，以海外销售管理需求为主。 \n3. 新能源车产业链：\n从年初的缺芯到补贴退坡，从传统车企入局到新势力造车格局变化，从特\n斯拉涨价到超级工厂停产降价，一转眼今天上市的车型已经全面刷新软硬\n技术，一年半之前还是香饽饽的上一代车型已经被次代全面革新。\n2022年注定是新能源车及其产业链提前进入激烈竞争的一年， 既是技术\n产品营销的竞争，亦是人才争夺的竞争。这一年，国内新能源车的渗透率\n突破25%， 提前达到三年后目标。 \nKOS新能源招聘团队认为，在2022人才争夺战中，新能源车人才竞争的', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf', 'page': 43}),
 Document(page_content='2022\nNew Energy\n40\n2022年，在经济增长迟缓、国际关系冲突、国际贸易受阻、国内需求不\n振，国人饱受新冠困扰的背景下，新能源产业是少数几个被看好的持续发\n展行业之一。 \nKOS新能源招聘团队认为在过去一年， 新能源相关企业发展有三个焦点，\n分别为风电光伏集约式发展、储能板块爆发、新能源车产业链技术持续革\n新，而从招聘需求上，三个板块各有特点。\n1. 风电光电：\n由于实现双碳目标是我国新型电力系统全面铺开建设的重要动因。这一战\n略目标的实现，很大程度上取决于能源与电力的清洁化程度，以及数智化\n与分布式能源、智慧能源电力和清洁低碳能源开发利用等技术的应用。\n国家能源局数据显示，过去的一年，火电装机依然在国内占比50%以上。\n但在双碳战略的要求下，火电装机比重需逐年减少，新能源电力装机比重\n和供电能力需明显提高。与此同时，清洁能源消费占比持续提升，风电及', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf', 'page': 42}),
 Document(page

## 7. 文档问答

创建一个 prompt 模板，用于进行文档问答

需要让语言模型根据文档内容来进行回答，超出范围的内容则回答我不知道

In [28]:
from langchain.prompts import PromptTemplate

prompt_template="""请注意：请谨慎评估提问与提示的上下文材料的相关性，只根据本段输入文字信息的内容进行回答，如果提问与提供的上下文材料无关，请回答"我不知道"，不要回答无关答案或编造答案：
-----------------------
上下文: 
    {context}
-----------------------
提问: {query}
-----------------------
答案:"""

prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=prompt_template
)

### OpenAI 模型（二选一）

In [29]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

### 本地模型（二选一）

In [ ]:
from ChatGLM import ChatGLM

llm = ChatGLM(model_path='chatglm-6b-int4')

### Chain 编写

LangChain 自带了 load_qa_chain，可以将文档和 prompt 编写到一起，然后调用模型，串联起整个流程

chain 的参数：

第一个是模型类型，这里使用的是默认的 gpt-3.5-turbo

第二个是传递类型，stuff 是将 docs 全部一次性传给模型进行归纳

第三个是前面编写好的模板，还未指定具体的 context 和 query

In [30]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

调用 chain

In [31]:
result = chain({"input_documents": docs, "query": query}, 
               return_only_outputs=True)

In [32]:
result

{'output_text': '突破25%。'}

尝试其他问题

In [33]:
query = "关于2022年的机器人行业企业密度，各国的排名怎么样？"
docs = retriever.get_relevant_documents(query)

In [34]:
docs

[Document(page_content='2022\nRobotics\n22\n2022年是全球机器人产业高速发展的一年，同时也是中国机器人行业快速\n向全球市场进军的一年。\n在这一年里，国内机器人厂商攻克了控制器、伺服系统等关键核心零部件\n领域的难题，实现了核心零部件国产化。这不仅促进了机器人产业的加\n速，也帮助到“机器人+”等拓展了应用领域。\n去年，全球机器人市场规模达到513亿美元（折3576亿元人民币）。而国\n内机器人市场规模也突破了1213亿元人民币。在过往的5年里，国内机器\n人年均增长率为22%。2022年上半年机器人行业融资总额超315亿元人民\n币，其中，单笔融资金额达到20亿元人民币，其主要业务为轻量级协作机\n器人研发。\n国内很多机器人公司借助现有成熟工业链体系，完成弯道超车，在疫情期\n间，开始把业务重心移至国外。2022年，国内机器人公司出海业务比例占\n到30%，更有企业有超50%的业务来自海外市场。', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf', 'page': 24}),
 Document(page_content='2023\nRobotics\n24\n国际机器人联合会（IFR）2022年底统计显示，中国机器人行业企业密度\n跻身世界第五位——前四位分别是，韩国、新加坡、日本、德国，美国则\n排在第九。\n2023年，机器人行业市场预计将保持20到25%的增长，而市场规模则仍在\n1000亿人民币以上。\n推动工业机器人发展有三大底层驱动因素，分别为人口结构改变、用劳用\n工成本的增加、下游应用产业的突出需求。\n而工业机器人产业链主要由零部件（上游）、机器人本体（中游）、系统\n集成（下游）以及行业应用等几个环节组成。在成本结构中，上游技术壁\n垒较高的核心零部件占比较高，伺服系统、控制器与减速器成本占比超\n70%，其中减速器占到一半，本体制造约占15%。\n近年来，随着工业机器人、高端数控机床等智能制造领域的快速发展，谐\n波与RV（旋转矢量）减速器已成为高精密传动领域广泛使用的精密减速', metadata={'source': 'KOS：2023中国市场招聘趋势.pdf', 'page': 26}),
 Document(page_content='器人研

不需要重复创建 chain，只需要调用即可

In [35]:
result = chain.run(input_documents = docs, query = query)

In [36]:
result

'中国机器人行业企业密度排名世界第五，前四位分别是韩国、新加坡、日本、德国，美国排名第九。'